#Optimisation_portfolios

In [190]:
import numpy as np
import cvxpy as cp
import os
import pandas as pd
from numpy.linalg import inv
from numpy.linalg import eig
from scipy import optimize 
from scipy.optimize import minimize

In [191]:
#the value of the optimal weight_ MVO Markowitz
def markowitz_optimization(Sigma, mu, lmbda):
    n = len(mu) # number of assets

    # Define variables
    w = cp.Variable(n) # portfolio weights

    # Define objective function
    objective = cp.Maximize(mu.T @ w - lmbda * cp.quad_form(w, Sigma))

    # Define constraints
    constraints = [cp.sum(w) == 1, w >= 0] # fully invested, long-only portfolio

    # Solve problem
    problem = cp.Problem(objective, constraints)
    problem.solve()

    # Return results
    w_opt = w.value
    return_opt = mu.T @ w_opt
    risk_opt = np.sqrt(w_opt.T @ Sigma @ w_opt)

    return w_opt, return_opt, risk_opt


In [192]:
"""testing the method"""
# Define the covariance matrix
Sigma = np.array([[0.05, 0.03, 0.02],
                  [0.03, 0.10, 0.05],
                  [0.02, 0.05, 0.08]])

# Define the mean return vector
mu = np.array([0.1, 0.2, 0.15])

# Define the regularization parameter
lamda = 0.1

# Call the markowitz_optimization function
w_opt, return_opt, risk_opt = markowitz_optimization(Sigma, mu, lamda)

# Print the optimal portfolio weights, return, and risk
print("Optimal portfolio weights:")
print(w_opt)
print("Expected return of the optimal portfolio:")
print(return_opt)
print("Risk of the optimal portfolio:")
print(risk_opt)


Optimal portfolio weights:
[ 1.10128082e-25  1.00000000e+00 -2.76397693e-23]
Expected return of the optimal portfolio:
0.2
Risk of the optimal portfolio:
0.31622776601683794


In [193]:
#the risk-based weight 


def WRiskBased(sigma):
   
    nb = sigma.shape[0]
    w = sigma @ np.ones(nb) / (np.ones(nb).T @ sigma @ np.ones(nb))
    return w

mv = WRiskBased(inv(sigma))
ew = WRiskBased(np.diag(np.ones(4)))
erv = WRiskBased(inv(np.diag(np.diag(sigma))))
erb = WRiskBased(inv(np.diag(np.sqrt(np.diag(sigma)))))
pd.DataFrame([mv,ew,erv,erb])


,0,1,2,3
0,0.666667,0.333333,0.000000,NaN
1,0.250000,0.250000,0.250000,0.25
2,0.405405,0.270270,0.324324,NaN
3,0.368878,0.301188,0.329934,NaN


In [194]:
sigma = np.array([[1, 0.5, 0.2],
                  [0.5, 2, 0.7],
                  [0.2, 0.7, 0.5]])

WRiskBased(sigma)


array([0.26984127, 0.50793651, 0.22222222])

In [195]:
#the optimal solution of robust optmization

def WROB(mu,sigma,sqrt_omega,kappa,gamma):
    nb = sigma.shape[0]
    w = cp.Variable(nb)
    ret = mu.T @ w
    risk = cp.quad_form(w,sigma) + cp.quad_form(sqrt_omega @ w,np.diag(kappa))
    prob = cp.Problem(cp.Maximize(ret - gamma*risk),[cp.sum(w) == 1, w >= 0])
    prob.solve()
    return w.value

In [196]:
"""testing the method"""
# Define inputs
mu = np.array([0.1, 0.2, 0.15])
sigma = np.array([[0.2, 0.1, 0.15], [0.1, 0.3, 0.2], [0.15, 0.2, 0.25]])
sqrt_omega = np.array([[0.1, 0, 0], [0, 0.2, 0], [0, 0, 0.3]])
kappa = np.array([0.1, 0.2, 0.3])
gamma = 0.5

# Call WROB function
result = WROB(mu, sigma, sqrt_omega, kappa, gamma)

# Print result
print(result)


[0.33589019 0.63746468 0.02664514]


In [197]:
def optimize_portfolio_VaR(mu, Sigma, alpha):
    n = len(mu)
    w = cp.Variable(n)
    ret = mu.T @ w
    L = np.linalg.cholesky(Sigma)
    # Compute VaR at alpha level
    z = cp.Variable()
    prob = cp.Problem(cp.Maximize(ret - z),
                      [cp.sum(w) == 1, w >= 0, 
                       cp.norm(L @ w, 2) <= z * np.sqrt(n-1), 
                       cp.norm(w, 1) <= 1])
    prob.solve()
    return w.value

In [198]:
mu = np.array([0.1, 0.2, 0.15])
Sigma = np.array([[0.05, 0.03, 0.02], 
                  [0.03, 0.10, 0.05], 
                  [0.02, 0.05, 0.08]])
alpha = 0.05 # VaR level
w = optimize_portfolio_VaR(mu, Sigma, alpha)
print(w)

[4.29644933e-09 4.28227577e-01 5.71772418e-01]


In [199]:



def robust_portfolio_optimization(mu, Sigma, kappa, lamda):
    n = len(mu) # number of assets
    Omega = np.linalg.inv(Sigma) # compute Omega as the inverse of Sigma
    
    def obj_fun(w):
        w = np.array(w)
        w_T_Omega_w = np.sqrt(w.dot(Omega).dot(w))
        return -mu.dot(w) + kappa * w_T_Omega_w - lamda * w_T_Omega_w ** 2
    
    # set up optimization problem
    constraints = [{'type': 'eq', 'fun': lambda w: np.sum(w) - 1},
                   {'type': 'ineq', 'fun': lambda w: w}]
    bounds = tuple([(0, None) for i in range(n)])
    
    # solve optimization problem
    w0 = np.ones(n) / n # starting point for optimization
    result = minimize(obj_fun, w0, bounds=bounds, constraints=constraints)
    
    return result.x


In [200]:
#Define the inputs
mu = np.array([0.05, 0.07, 0.10])
Sigma = np.array([[0.06, 0.02, 0.01], [0.02, 0.09, 0.03], [0.01, 0.03, 0.10]])
kappa = 0.05
lamda = 0.1

#Call the function
robust_portfolio_optimization(mu, Sigma, kappa, lamda)

array([1.00000000e+00, 2.02060590e-14, 2.77555756e-15])

In [201]:
#Maximizes the Sharpe Ratio of a portfolio to find the optimum weight

def MaximizeSharpeRatioOptmzn(mean_returns, covar_returns, risk_free_rate):


    portfolio_size = len(mean_returns)

    # Define maximization of Sharpe Ratio (max(f)=-min(-f))
    def f(x, mean_returns, covar_returns, risk_free_rate):
        Rf=risk_free_rate
        rx=np.matmul(np.array(mean_returns), x.T)
        sigma = np.sqrt(np.matmul(np.matmul(x, covar_returns), x.T)) 
        return -(rx-Rf / sigma)

    # Define equality constraint representing fully invested portfolio
    def constraint(x):
        return  np.matmul(np.ones(x.shape), x.T) - 1 
    
    # Define bounds and other parameters
    x_init = np.repeat(1 / portfolio_size, portfolio_size)
    cons = ({'type': 'eq', 'fun': constraint})
    lb = 0
    ub = 1
    bnds = tuple([(lb, ub) for x in x_init])
    
    # Invoke minimize solver
    opt = optimize.minimize(f, x0=x_init, args=(mean_returns, covar_returns, risk_free_rate),
                            method='SLSQP', bounds=bnds, constraints=cons, tol=10**-3)
    
    # Compute maximal Sharpe Ratio and optimal weights
    x_optimal = opt.x
    x_optimal_array = np.array(x_optimal)
    risk = np.matmul((np.matmul(x_optimal_array, covar_returns)), np.transpose(x_optimal_array))
    exp_return = np.matmul(np.array(mean_returns), x_optimal_array.T)
    ann_risk = np.sqrt(risk * 251) 
    ann_ret = 251 * np.array(exp_return) 
    max_sharpe_ratio = (ann_ret - risk_free_rate) / ann_risk 

    return x_optimal_array, max_sharpe_ratio


In [202]:
# Sample inputs
mean_returns = [0.1, 0.2, 0.15]
covar_returns = np.array([[0.05, 0.03, 0.02], [0.03, 0.06, 0.04], [0.02, 0.04, 0.08]])
risk_free_rate = 0.03

# Call function
optimal_weights, sharpe_ratio = MaximizeSharpeRatioOptmzn(mean_returns, covar_returns, risk_free_rate)

# Print results
print('Optimal weights (%):\n', optimal_weights * 100)
print('Maximal Sharpe Ratio:', sharpe_ratio)


Optimal weights (%):
 [ 0.         58.31756419 41.68243581]
Maximal Sharpe Ratio: 12.234592941577718


In [203]:
data = pd.read_excel("/Users/macbook/Desktop/DataProjets.xlsx", "MarketCaps", index_col="Date")

def clean_data(data_set):
    
    #remove the NaN values as follow : 
    # - if there is no price for a day, replace the value whith the price of the previous day
    # - if the company has never issued a prize or if it has gone bankrupt: keep the value NaN
    
    clean_dataframe = pd.DataFrame(columns=data_set.columns, index=data_set.index)
    for tickers in data_set.columns:
        
        x = data_set[tickers].last_valid_index()
        clean_dataframe[tickers][:x] = data_set[tickers][:x].fillna(method ='ffill', limit=10)
        
    return clean_dataframe

In [204]:
data.head()

,200001,200247,200597,200784,200815,201160,201947,201995,202045,BYNF41,...,BYVYWS,BD0Q55,BDSFG9,BYPZJQ,BHXCF8,BK73B4,BJ1F30,BK4N0D,BK531S,BMWS3X
Date,,,,,,,,,,,,,,,,,,,,,
2002-12-31,0.0,0.000304,0.021304,0.000290,0.001006,0.001518,0.0,0.003986,0.003233,0.002689,...,0.0,0.0,0.0,0.000455,0.0,0.0,0.0,0.0,0.0,0.0
2003-01-31,0.0,0.000304,0.021304,0.000290,0.001006,0.001518,0.0,0.003986,0.003233,0.002689,...,0.0,0.0,0.0,0.000455,0.0,0.0,0.0,0.0,0.0,0.0
2003-02-28,0.0,0.000295,0.021539,0.000308,0.001062,0.001440,0.0,0.003635,0.003073,0.002830,...,0.0,0.0,0.0,0.000430,0.0,0.0,0.0,0.0,0.0,0.0
2003-03-31,0.0,0.000319,0.021456,0.000344,0.001153,0.001524,0.0,0.003774,0.002958,0.002651,...,0.0,0.0,0.0,0.000410,0.0,0.0,0.0,0.0,0.0,0.0
2003-04-30,0.0,0.000485,0.021279,0.000379,0.001184,0.001451,0.0,0.003940,0.002994,0.002873,...,0.0,0.0,0.0,0.000459,0.0,0.0,0.0,0.0,0.0,0.0


In [141]:
clean_data(data).head()

,200001,200247,200597,200784,200815,201160,201947,201995,202045,BYNF41,...,BYVYWS,BD0Q55,BDSFG9,BYPZJQ,BHXCF8,BK73B4,BJ1F30,BK4N0D,BK531S,BMWS3X
Date,,,,,,,,,,,,,,,,,,,,,
2002-12-31,0.0,0.000304,0.021304,0.00029,0.001006,0.001518,0.0,0.003986,0.003233,0.002689,...,0.0,0.0,0.0,0.000455,0.0,0.0,0.0,0.0,0.0,0.0
2003-01-31,0.0,0.000304,0.021304,0.00029,0.001006,0.001518,0.0,0.003986,0.003233,0.002689,...,0.0,0.0,0.0,0.000455,0.0,0.0,0.0,0.0,0.0,0.0
2003-02-28,0.0,0.000295,0.021539,0.000308,0.001062,0.00144,0.0,0.003635,0.003073,0.00283,...,0.0,0.0,0.0,0.00043,0.0,0.0,0.0,0.0,0.0,0.0
2003-03-31,0.0,0.000319,0.021456,0.000344,0.001153,0.001524,0.0,0.003774,0.002958,0.002651,...,0.0,0.0,0.0,0.00041,0.0,0.0,0.0,0.0,0.0,0.0
2003-04-30,0.0,0.000485,0.021279,0.000379,0.001184,0.001451,0.0,0.00394,0.002994,0.002873,...,0.0,0.0,0.0,0.000459,0.0,0.0,0.0,0.0,0.0,0.0


In [148]:

import yfinance as yf
import calendar

# Load the mapping file
mapping_df =  pd.read_excel("/Users/macbook/Desktop/DataProjets.xlsx", "Mapping")

# Load the MarketCaps file
market_caps_df =  pd.read_excel("/Users/macbook/Desktop/DataProjets.xlsx", "MarketCaps")

#Download the daily data of the companies
finance_info = yf.download(mapping_df["Tickers"].to_list(), start=MarketCaps.index[0], end=MarketCaps.index[-1])
    
Prices = finance_info["Close"]
    
#Save the datas
Prices.to_csv("Prices_data.csv")


[*********************100%***********************]  561 of 561 completed

37 Failed downloads:
- MXIM: No timezone found, symbol may be delisted
- RRD: No timezone found, symbol may be delisted
- MDP: No data found for this date range, symbol may be delisted
- CTXS: No timezone found, symbol may be delisted
- VAR: No timezone found, symbol may be delisted
- WLTW: No timezone found, symbol may be delisted
- NEBLQ: No timezone found, symbol may be delisted
- XLNX: No timezone found, symbol may be delisted
- CXO: No timezone found, symbol may be delisted
- DISCA: No timezone found, symbol may be delisted
- NAV: No timezone found, symbol may be delisted
- SHLDQ: No timezone found, symbol may be delisted
- FTRCQ: No timezone found, symbol may be delisted
- VALPQ: No timezone found, symbol may be delisted
- DOFSQ: No timezone found, symbol may be delisted
- FLIR: No timezone found, symbol may be delisted
- TIF: No timezone found, symbol may be delisted
- NLOK: No timezone found, symbol may b

In [151]:
returns_df= Prices.pct_change().drop(Prices.index[0])

In [154]:

mu = returns_df.mean().to_numpy()

In [155]:
mu

array([ 6.94990255e-04,  4.34310506e-04,  8.42823880e-04,  6.49581113e-04,
        1.47762676e-03,  7.09363356e-04,  6.27692650e-04,  1.44004371e-03,
        6.85102335e-04,  9.03245300e-04,  5.86528790e-04,  5.58135679e-04,
        4.95208668e-04,             nan,  9.65877496e-04,  2.40650838e-04,
        3.33672986e-04,  7.21137135e-04,  7.66654924e-04,  5.67633051e-04,
        1.33875306e-04,  4.08337191e-04,  5.59164652e-04,  4.82321478e-04,
        1.22955451e-03,  8.67454734e-04,  1.43762055e-03,  4.20896780e-04,
                   nan,  6.93923013e-04,  2.81633772e-04,  1.13132169e-03,
        7.79225231e-04,  6.30828185e-04,  4.58481973e-04,  8.54769174e-04,
        9.92240157e-04,  1.22468785e-03,  7.25810259e-04,  1.18148674e-03,
        1.38711883e-03,  9.95307541e-04,             nan,  6.82999416e-04,
        5.66887447e-04,  5.45430465e-04,  8.65330260e-04,  9.18793313e-04,
        4.58709054e-04,  6.86893321e-04,  9.75218264e-04,  9.89314183e-04,
        5.00227008e-04,  

In [165]:
mu_updated=clean_data(pd.DataFrame(mu)).to_numpy()
mu_updated

array([[0.0006949902549800613],
       [0.0004343105057662014],
       [0.0008428238801991018],
       [0.0006495811131313338],
       [0.0014776267641183269],
       [0.0007093633562545911],
       [0.0006276926496177051],
       [0.0014400437118068314],
       [0.0006851023348752705],
       [0.0009032453000626095],
       [0.0005865287900023248],
       [0.0005581356789546647],
       [0.0004952086684889389],
       [0.0004952086684889389],
       [0.0009658774957100666],
       [0.00024065083824656843],
       [0.00033367298615337345],
       [0.0007211371349740006],
       [0.0007666549236635759],
       [0.0005676330512222715],
       [0.00013387530618280317],
       [0.0004083371906522743],
       [0.0005591646523461393],
       [0.000482321477923343],
       [0.0012295545130941003],
       [0.0008674547342179753],
       [0.0014376205452837514],
       [0.00042089678005845536],
       [0.00042089678005845536],
       [0.0006939230128165161],
       [0.0002816337721491652],
    

In [171]:
cov_matrix = returns_df.cov().to_numpy()

In [173]:
cov_matrix
def make_symmetric(matrix):
    if np.allclose(matrix, matrix.T):
        return matrix
    else:
        return (matrix + matrix.T) / 2

In [174]:
markowitz_optimization(make_symmetric(cov_matrix), mu, 0.5)

ValueError: Quadratic form matrices must be symmetric/Hermitian.